In [ ]:
import ee
import geemap
import logging
import multiprocessing
import os
import torch
import pandas as pd
from skimage.measure.entropy import shannon_entropy
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import json
from utils.downloading import getResult, getRequests, get_files_info

In [ ]:
GEE_PROJECT_ID = "fusion-371234"
ee.Authenticate()
ee.Initialize(project=GEE_PROJECT_ID)

In [ ]:
Map = geemap.Map()
Map = geemap.Map(center=(-5.1841285, -37.3477805), zoom=12)

Map.add_basemap("HYBRID")
Map

In [ ]:
# Convert to GeoJSON format
geojson_dict = Map.user_roi.getInfo()

# Save to file
with open("ROI.json", "w") as f:
    json.dump(geojson_dict, f)

print("✅ Region saved as ROI.json")

In [ ]:
# Read the JSON geometry
with open("ROI.json", "r") as f:
    region_data = json.load(f)

# Convert to Earth Engine Geometry
region = ee.Geometry(region_data)

In [ ]:
S2_SR_COLLECTION_ID = "COPERNICUS/S2_SR_HARMONIZED"
selected_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
CLOUD_FILTER = 100
PROCESSES = multiprocessing.cpu_count() - 1
dimensions = "256x256"
buffer = (int(dimensions.split('x')[0]) / 2) * 10 
crs = "EPSG:4326"
num_samples = 10
DATE = ["2024-06-01", "2024-06-14"]

params = {
    "count": num_samples,  # How many image sample points to export
    "buffer": buffer,  # The buffer distance (m) around each point
    "scale": 10,  # The scale to do stratified sampling
    "bands": None, # The bands to export
    "seed": 1,  # A randomization seed to use for subsampling.
    "dimensions": dimensions,  # The dimension of each image chip
    "format": "GEO_TIFF",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    "prefix": "sample_",  # The filename prefix
    "processes": min(25, PROCESSES),  # How many processes to used for parallel processing
    "out_dir": "training",  # The output directory. Default to the current working directly
    "crs": crs,  # The CRS to use for the output image
}

s2_sr_col = ee.ImageCollection(S2_SR_COLLECTION_ID).filterBounds(region).filterDate(DATE[0], DATE[1])

params['bands'] = [{
    'id': band, 
    'scale': (
        10 if band in ['B2', 'B3', 'B4', 'B8'] else 
        20 if band in ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'] else 
        60
    ), 
    'crs': params['crs']} for band in selected_bands]


In [ ]:
images = []
samples = []
col_size = s2_sr_col.size().getInfo()
print(f"Number of images: {col_size}")
for i in range(col_size):
    sr_img = s2_sr_col.toList(s2_sr_col.size()).getInfo()[i]
    id = sr_img['properties']['PRODUCT_ID']
    sr_img = s2_sr_col.filter(ee.Filter.eq("PRODUCT_ID", id)).first()
    sr_img = sr_img.select(selected_bands)
    images.append(sr_img)
    samples.append(id)

os.makedirs(params['out_dir'], exist_ok=True)
for i in range(len(images)):
    image = images[i]
    id = samples[i]
    logging.basicConfig()
    items = getRequests(params, image, region)
    pool = multiprocessing.Pool(params["processes"])
    pool.starmap(getResult, [(index, item, image, params, id) for index, item in enumerate(items)])
    pool.close()      

In [ ]:
n_files, name_files = get_files_info(params["out_dir"])
name_files = sorted(name_files)

for i in range(n_files):
    



Total number of files: 31
README.md
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_00.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_01.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_02.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_03.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_04.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_05.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_06.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_07.tif
S2A_MSIL2A_20240608T125311_N0510_R052_T24MXV_20240608T171205_sample_08.tif
